# Project Set Up

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import torch
from peft import PeftModel

## Load Test Dataset

In [3]:
from read_data import read_test_data, convert_to_qwen_format_user, read_train_original_data

In [4]:
Math_dataset = read_train_original_data("maths",n_samples=500)  # specify n_samples here

Subsampled to 500 examples for maths dataset.


In [5]:
# MedQA_dataset = read_train_original_data("medqa",n_samples=500)  # specify n_samples here

In [6]:
Math_dataset.iloc[0]['question']

'Solve this math problem step by step. Put your final answer in \\boxed{}.\n\nProblem: If $4^6=8^n$, what is $n$?'

In [7]:
test_formatted_question = convert_to_qwen_format_user(Math_dataset.iloc[11]['question'])
# test_formatted_question = convert_to_qwen_format_user(MedQA_dataset.iloc[0]['question'])

In [8]:
print(test_formatted_question)

<|im_start|>system
You are helpful assistant for mathematical problems. Please reason step by step, and put your final answer within \boxed{}.
# Rules:
Rule 1. If you are sure about your final answer, indicate your confidence by appending the token <|c_math|>.
Rule 2. If you are unsure about your final answer, indicate your uncertainty by appending the token <|u_math|>.
<|im_end|>
<|im_start|>user
Solve this math problem step by step. Put your final answer in \boxed{}.

Problem: The first $20$ numbers of an arrangement are shown below. What would be the value of the $40^{\mathrm{th}}$ number if the arrangement were continued?

$\bullet$ Row 1: $2,$ $2$

$\bullet$ Row 2: $4,$ $4,$ $4,$ $4$

$\bullet$ Row 3: $6,$ $6,$ $6,$ $6,$ $6,$ $6$

$\bullet$ Row 4: $8,$ $8,$ $8,$ $8,$ $8,$ $8,$ $8,$ $8$<|im_end|>
<|im_start|>assistant



# Run VLLM_PEFT_MODEL

In [9]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from transformers import AutoTokenizer

INFO 11-19 00:44:43 [__init__.py:216] Automatically detected platform cuda.


In [10]:
tensor_parallel_size = 1  # Adjust based on available GPUs
model_path = "Qwen/Qwen2.5-Math-1.5B-Instruct"


In [11]:
llm = LLM(model=model_path, enable_lora=True, tensor_parallel_size=tensor_parallel_size)

INFO 11-19 00:44:46 [utils.py:328] non-default args: {'disable_log_stats': True, 'enable_lora': True, 'model': 'Qwen/Qwen2.5-Math-1.5B-Instruct'}


INFO 11-19 00:44:54 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-19 00:44:54 [__init__.py:1815] Using max model len 4096
INFO 11-19 00:44:55 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 11-19 00:44:55 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
(EngineCore_DP0 pid=3609380) INFO 11-19 00:44:56 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=3609380) INFO 11-19 00:44:56 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='Qwen/Qwen2.5-Math-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=Fals

[W1119 00:44:58.556431689 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=3609380) INFO 11-19 00:44:59 [gpu_model_runner.py:2370] Loading model from scratch...
(EngineCore_DP0 pid=3609380) INFO 11-19 00:44:59 [cuda.py:362] Using Flash Attention backend on V1 engine.
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:00 [weight_utils.py:348] Using model weights format ['*.safetensors']
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:00 [weight_utils.py:406] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:00 [default_loader.py:268] Loading weights took 0.42 seconds
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:00 [punica_selector.py:19] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:01 [gpu_model_runner.py:2392] Model loading took 2.9145 GiB and 1.671628 seconds
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:08 [backends.py:539] Using cache directory: /home/long_2/.cache/vllm/torch_compile_cache/d4bbb48c97/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:08 [backends.py:550] Dynamo bytecode transform time: 6.20 s
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:11 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.203 s
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:12 [monitor.py:34] torch.compile takes 6.20 s in total
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:13 [gpu_worker.py:298] Available KV cache memory: 35.57 GiB
(EngineCore_DP0 p

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:05<00:00, 12.06it/s]


(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:20 [gpu_model_runner.py:3118] Graph capturing finished in 6 secs, took 3.03 GiB
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:20 [gpu_worker.py:391] Free memory on device (44.05/44.35 GiB) on startup. Desired GPU memory utilization is (0.9, 39.92 GiB). Actual usage is 2.91 GiB for weight, 1.41 GiB for peak activation, 0.02 GiB for non-torch memory, and 3.03 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=34783182643` to fit into requested memory, or `--kv-cache-memory=39217859584` to fully utilize gpu memory. Current kv cache memory in use is 38197346099 bytes.
(EngineCore_DP0 pid=3609380) INFO 11-19 00:45:20 [core.py:218] init engine (profile, create kv cache, warmup model) took 18.78 seconds
INFO 11-19 00:45:21 [llm.py:295] Supported_tasks: ['generate']
INFO 11-19 00:45:21 [__init__.py:36] No IOProcessor plugins requested by the model


In [12]:
# saved_peft_model_path = "Qwen2.5-Math-1.5B_Confident_Task_LoRA_6750_final"
saved_peft_model_path = "Qwen2.5-Math-1.5B-Instruct_Confident_Task_LoRA_6750_final"


In [18]:
sampling_params = SamplingParams(
    temperature=0.0,  # Deterministic generation for evaluation
    top_p=1.0,
    max_tokens=4086,
    stop=["</s>", "<|im_end|>", "<|c_math|>", "<|u_math|>"],
    include_stop_str_in_output=True
)

In [19]:
lora_request = None
lora_path = f"finetuned_models/{saved_peft_model_path}"
lora_request = LoRARequest("confident_task_lora_adapter", 1, lora_path)

## Test with Math Dataset

In [24]:
test_formatted_question = convert_to_qwen_format_user(Math_dataset.iloc[9]['question'])
# test_formatted_question = convert_to_qwen_format_user(MedQA_dataset.iloc[0]['question'])
prompts = [test_formatted_question]
print(test_formatted_question)

<|im_start|>system
You are helpful assistant for mathematical problems. Please reason step by step, and put your final answer within \boxed{}.
# Rules:
Rule 1. If you are sure about your final answer, indicate your confidence by appending the token <|c_math|>.
Rule 2. If you are unsure about your final answer, indicate your uncertainty by appending the token <|u_math|>.
<|im_end|>
<|im_start|>user
Solve this math problem step by step. Put your final answer in \boxed{}.

Problem: Coach Grunt is preparing the 5-person starting lineup for his basketball team, the Grunters.  There are 12 players on the team.  Two of them, Ace and Zeppo, are league All-Stars, so they'll definitely be in the starting lineup.  How many different starting lineups are possible?  (The order of the players in a basketball lineup doesn't matter.)<|im_end|>
<|im_start|>assistant



In [25]:
outputs = llm.generate(prompts, sampling_params, lora_request=lora_request)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [26]:
print(outputs[0].outputs[0].text.strip())

To determine the number of different starting lineups possible for Coach Grunt's basketball team, we need to follow these steps:

1. **Identify the total number of players and the number of players to be selected:**
   - Total players on the team: 12
   - Players to be in the starting lineup: 5
   - Two players (Ace and Zeppo) are definitely in the lineup.

2. **Calculate the number of remaining players to be selected:**
   - Since Ace and Zeppo are already in the lineup, we need to select 3 more players from the remaining players.
   - Number of remaining players: \( 12 - 2 = 10 \)

3. **Determine the number of ways to choose 3 players from the remaining 10:**
   - This is a combination problem because the order in which the players are selected does not matter.
   - The number of ways to choose 3 players from 10 is given by the combination formula:
     \[
     \binom{n}{r} = \frac{n!}{r!(n-r)!}
     \]
     where \( n \) is the total number of items to choose from, and \( r \) is th

## Test with Medical Dataset

In [59]:
test_formatted_question = convert_to_qwen_format_user(MedQA_dataset.iloc[10]['question'])
tests_formatted_question_1 = convert_to_qwen_format_user(MedQA_dataset.iloc[1]['question'])
# test_formatted_question = convert_to_qwen_format_user(MedQA_dataset.iloc[0]['question'])
prompts = [test_formatted_question, tests_formatted_question_1]
print(test_formatted_question)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful and precise assistant.
At the end of each answer, append exactly one token:
- Medicine: <C_MED> if confident, <U_MED> if uncertain.
- Math: <C_MATH> if confident, <U_MATH> if uncertain.<|im_end|>
<|im_start|>user
Answer the following medical question by selecting the correct option.

Question: A 46-year-old woman comes to the physician because of a 2-week history of diplopia and ocular pain when reading the newspaper. She also has a 3-month history of amenorrhea, hot flashes, and increased sweating. She reports that she has been overweight all her adult life and is happy to have lost 6.8-kg (15-lb) of weight in the past 2 months. Her pulse is 110/min, and blood pressure is 148/98 mm Hg. Physical examination shows moist palms and a nontender thyroid gland that is enlarged to two times its normal size. Ophthalmologic examination shows prominence of the globes of the eyes, bilateral lid retraction, conjunctival i

In [61]:
outputs = llm.generate(prompts, sampling_params, lora_request=lora_request)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [62]:
full_preds = [output.outputs[0].text.strip() for output in outputs]

In [64]:
for pred in full_preds:
    print(pred)
    print("-----")

C: Glycosaminoglycan accumulation in the orbit

Explanation: The patient's symptoms of diplopia, ocular pain, and lid retraction are most consistent with a condition known as orbital myxoma, which is a benign tumor composed of glycosaminoglycans. These glycosaminoglycans can accumulate in the orbit and cause the symptoms described. The other options are less likely given the patient's clinical presentation and the presence of a nontender thyroid gland and enlarged thyroid, which are more indicative of hyperthyroidism. The fine resting tremor and increased reflexes are also more consistent with hyperthyroidism. The other options are less likely given the patient's clinical presentation and the presence of a nontender thyroid gland and enlarged thyroid, which are more indicative of hyperthyroidism. The fine resting tremor and increased reflexes are also more consistent with hyperthyroidism. The other options are less likely given the patient's clinical presentation and the presence of a 

In [65]:
decoded_preds = [i.strip().split('\n')[0][0] for i in full_preds]

In [66]:
decoded_preds

['C', 'A']